In [87]:
# !pip install boto3

  Obtaining dependency information for boto3 from https://files.pythonhosted.org/packages/11/df/4e09f629ad5aa710e22be0daf767381a4bb7306416d3d255d7bc9a3920ed/boto3-1.34.85-py3-none-any.whl.metadata
  Obtaining dependency information for botocore<1.35.0,>=1.34.85 from https://files.pythonhosted.org/packages/4e/00/60de4c95e406e399f47be280ce87110d752300bfe8e584876e4db16383e0/botocore-1.34.85-py3-none-any.whl.metadata
  Obtaining dependency information for s3transfer<0.11.0,>=0.10.0 from https://files.pythonhosted.org/packages/83/37/395cdb6ee92925fa211e55d8f07b9f93cf93f60d7d4ce5e66fd73f1ea986/s3transfer-0.10.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 18.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 10.8 MB/s eta 0:00:00
  Attempting uninstall: botocore
    Found existing installation: botocore 1.29.76
    Uninstalling botocore-1.29

In [100]:
# !pip install python-dotenv

In [156]:
import requests
import json
import datetime as dt
import boto3
from dotenv import load_dotenv
import os
import logging
from botocore.exceptions import ClientError

load_dotenv()

True

In [149]:
def get_weather(date: str) -> object:
    API_KEY = '54ab2e260fc248557172cbca7ddcd23a'
    dailyAggEndpoint = 'https://api.openweathermap.org/data/3.0/onecall/day_summary'
    lat = '50.842589'
    lon = '-0.812001'
    units = 'metric'

    # Endpoint has weather data for a particular date from 1979-01-02 and long-term forecast for 1.5 years ahead
    url = f'{dailyAggEndpoint}?appid={API_KEY}&date={date}&lat={lat}&lon={lon}&units={units}'
    
    r = requests.get(url)
    
    if (r.status_code == 200):
        return json.loads(r.text)
    
    return {}

In [154]:
def dates_in_period(start_date: str, end_date: str = False) -> list:
    
    if not end_date:
        end_date = start_date
    
    # Dates inclusive
    start_date = dt.date.fromisoformat(start_date)
    end_date = dt.date.fromisoformat(end_date)
    
    dateDiff = end_date - start_date

    output = []

    for x in range (dateDiff.days + 1):
        date = start_date + dt.timedelta(days=x)
        output.append(date.isoformat())
        
    return output

In [110]:
def upload_file(s3_client: object, file_name: str, bucket: str, object_name: str = None, layer: str = 'raw') -> bool:

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = os.path.basename(file_name)

    object_name = f'{layer}/{object_name}'

    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [140]:
def upload_json(s3_client: object, obj: object, bucket: str, object_name: str, folder: str = 'weather') -> bool:

    object_name = f'{folder}/{object_name}'

    try:
        response = s3_client.put_object(Body = bytes(json.dumps(obj, indent=2).encode('UTF-8')), Bucket = bucket, Key = object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [161]:
def upload_weather_data(from_date: str, to_date: str = False) -> bool:

    if not to_date:
        to_date = from_date

    file_count = 0

    for date in dates_in_period(from_date, to_date):
        body = get_weather(date)

        if not body:
            raise Exception(f'error loading weather data for {date}')
        
        bucket = 'ab-rainfall-proj'

        s3 = boto3.client('s3')
        
        path = f'openweather_dailyagg_{date}.json'

        upload_json(s3, body, bucket, path, 'weather')
        
        file_count += 1
        
    print(f'saved and uploaded {file_count} files')
    
    return True

In [170]:
upload_weather_data('2014-01-01', '2014-12-31')

saved and uploaded 365 files


True